In [1]:
# -*- coding: utf-8 -*-
from tkinter import *
from time import *
import threading
import random

In [2]:
from qiskit import *

In [3]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

In [4]:
qc = QuantumCircuit(30, 30)

In [5]:
#number of successful whacks
whacks = 0
#number of the hole whacked, no longer needed
hole = 0

gate = -1
isControl = True

#screen elements
screen = Tk()
screen.config(bg="#62C370", width=500, height=520)
screen.resizable(0, 0)
screen.attributes('-topmost', True)
tip = Label(bg='steel blue', fg='green', text="Play with mouse for best experience")
tip.place(x=0, y=497)
lab = Label(text="WHAC-A-MOLE!", font=("Algerian", 30), bg='#3DA5D9', fg='#fff')
lab.place(x=110, y=0)
scorelabel = Label(text=whacks, width=7, font=('Bahnschrift', 30), bg='#FEC601')
scorelabel.place(x=170, y=52)
remark = Label(text="", width=17, bg='#EA7317', fg='#fff', font="Centaur 20")
remark.place(x=130, y=107)

In [6]:
def applyH():
    global gate
    gate = 0

def applyX():
    global gate
    gate = 1

def applyCX():
    global gate
    gate = 2

def applyZ():
    global gate
    gate = 3

hGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='h-gate',
    state='normal',
    command=applyH)
hGate.place(x = 421, y =160)

xGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='x-gate',
    state='normal',
    command=applyX)
xGate.place(x = 421, y =225)

cnotGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='CNOT',
    state='normal',
    command=applyCX)
cnotGate.place(x = 421, y =290)

zGate = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='z-gate',
    state='normal',
    command=applyZ)
zGate.place(x = 421, y =355)

In [7]:
def whac_a_mole():
    ready_set_whack()
    global hole
    for i in range(0, 30):
        #hole = random.randint(0, 34)
        #buttons[hole].config(
                #bg='pink',
                #fg='black')
        sleep(1)
        #buttons[hole].config(state='disabled', text='', bg='black')
       
    replaybtn.config(state='normal')
    global whacks
    if whacks < 5:
        remark.config(text="POOR")
    elif whacks >= 10 and whacks < 15:
        remark.config(text="BAD")
    elif whacks >= 15 and whacks < 20:
        remark.config(text="DO BETTER!")
    elif whacks >= 25 and whacks < 30:
        remark.config(text="GOOD")
    elif whacks >= 30 and whacks < 35:
        remark.config(text="GREAT!")
    elif whacks >= 35 and whacks < 40:
        remark.config(text="AWESOME!")
    elif whacks == 45:
        remark.config(text="WHACKED 'em ALL!", font="Centaur 18")

def start():
    global whacks
    whacks = 0
    global gate
    gate = -1
    remark.config(text="")
    replaybtn.config(state='disabled')
    hGate.config(state="disabled")
    xGate.config(state="disabled")
    zGate.config(state="disabled")
    cnotGate.config(state="disabled")
    t = threading.Thread(target=whac_a_mole)
    t.start()



In [8]:
startButton = Button(
    width=6,
    height=3,
    bg='ghost white',
    text='start!',
    state='normal',
    command=start)
startButton.place(x=19, y=90)

replaybtn = Button(text="Play Again", command=start)
replaybtn.place(x=0, y=2)

In [9]:
def rgbtohex(r,g,b):
    return f'#{r:02x}{g:02x}{b:02x}'

xPos = [19, 86, 153, 220, 287, 354]
#421
yPos = [160, 225, 290, 355, 419]

buttons = []

for xP in xPos:
    for yP in yPos:
        buttons.append(Button(
            width=6,
            height=3,
            bg=rgbtohex(255, 255, 255),
            text='【•】__【•】',
            state='normal',
            command=lambda m=len(buttons): onwhack(m)))
        buttons[len(buttons) - 1].place(x = xP, y = yP)

In [10]:
def onwhack(num):
    global gate
    if gate >= 0:
        placeGate(num, gate)
    else:
        global hole
        #add logic with quantum, onwhack = measurement
        buttons[num].config(state='disabled', text='*_*', fg='black')
        quantumMeasure(num)
        global whacks
        whacks += 1
        scorelabel.config(text="* " + str(whacks) + " *")
        print(gate)

In [11]:
gateList = ["【•】__【•】\n"] * 30

def placeGate(num, g):
    global gateList
    print(g)
    #H gate
    if g == 0:
        prevGate = g
        gateList[num] = gateList[num] + "H "
        buttons[num].config(bg='#80e8ff', text=gateList[num])
    #X gate
    if g == 1:
        prevGate = g
        gateList[num] = gateList[num] + "X "
        buttons[num].config(bg='#00d1ff', text=gateList[num])
    #CX gate
    if g == 2:
        prevGate = g
        gateList[num] = gateList[num] + "CX "
        buttons[num].config(bg='#ccf6ff', text=gateList[num])
    #Z gate
    if g== 3:
        prevGate = g
        gateList[num] = gateList[num] + "Z "
        buttons[num].config(bg='#e6faff', text=gateList[num])

In [12]:
def quantumMeasure(bnum):
    sim = Aer.get_backend('qasm_simulator')
    qexe = transpile(qc, sim)
    job = sim.run(qexe, shots = 10)
    result = job.result()
    #add measurement of qubit and set it to col, either 255 (0) or 0 (1)
    col = 0
    buttons[bnum].config(bg=rgbtohex(255, col, 255))

In [13]:
def ready_set_whack():
    sleep(1)
    scorelabel.config(text="Ready")
    sleep(1)
    scorelabel.config(text="Set")
    sleep(1)
    scorelabel.config(text="WHACK!!")
    sleep(1)
    scorelabel.config(text="0")
    for b in buttons: 
        b.config(state='normal')




In [ ]:

screen.mainloop()


1
1
2
3
-1
-1
-1
-1
-1
-1
